In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  5 07:55:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.5 MB/s eta 0:00:00


In [ ]:
import pickle
import torch
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F
import numpy as np

import math
import pandas as pd
import pip
import re

import time

import random

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/pakdd_model/full_dialog/final_combined_all_dialog.p", "rb") as f:
  final_combined_all_dialog = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/pakdd_model/full_dialog/disease_to_id_small.p", "rb") as f:
  disease_to_id_small = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/pakdd_model/full_dialog/new_dialog_id_list.p", "rb") as f:
  new_dialog_id_list = pickle.load(f)

In [ ]:
len(final_combined_all_dialog)

1367

In [ ]:
final_combined_all_dialog[0]

{'dialog': "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 'label': 'central retinal artery or vein occlusion'}

In [ ]:
disease_to_id_small

{'acanthosis nigricans': 0,
 'acariasis': 1,
 'acne': 2,
 'actinic keratosis': 3,
 'acute glaucoma': 4,
 'acute kidney injury': 5,
 'acute stress reaction': 6,
 'adhesive capsulitis of the shoulder': 7,
 'adjustment reaction': 8,
 'air embolism': 9,
 'alcohol intoxication': 10,
 'allergy': 11,
 'alzheimer disease': 12,
 'amyloidosis': 13,
 'amyotrophic lateral sclerosis als': 14,
 'ankylosing spondylitis': 15,
 'anxiety': 16,
 'aphakia': 17,
 'carbon monoxide poisoning': 18,
 'carcinoid syndrome': 19,
 'carpal tunnel syndrome': 20,
 'cat scratch disease': 21,
 'central retinal artery or vein occlusion': 22,
 'cerebral edema': 23,
 'chagas disease': 24,
 'chalazion': 25,
 'chancroid': 26,
 'chickenpox': 27,
 'chlamydia': 28,
 'chondromalacia of the patella': 29,
 'chronic back pain': 30,
 'chronic kidney disease': 31,
 'chronic pain disorder': 32,
 'complex regional pain syndrome': 33,
 'concussion': 34,
 'conductive hearing loss': 35,
 'conjunctivitis due to allergy': 36,
 'connective 

In [ ]:
len(new_dialog_id_list)

1367

In [ ]:
new_dialog_id_list[0]

1059

In [ ]:
tokenizer  = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")

In [ ]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
tokenizer.all_special_ids

[1, 3, 0, 2, 4]

In [ ]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
    }

tokenizer.add_special_tokens(p)
print(tokenizer.convert_tokens_to_ids('[DOC]'))
print(tokenizer.convert_tokens_to_ids('[PAT]'))
print(tokenizer.convert_tokens_to_ids('[SR_START]'))
print(tokenizer.convert_tokens_to_ids('[SR_END]'))
print(len(tokenizer))
print(bert_model.resize_token_embeddings(len(tokenizer)))
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]
30522
30523
30524
30525
30526
Embedding(30526, 768)
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
[1, 3, 0, 2, 4, 30522, 30523, 30524, 30525]


In [ ]:
tokenizer.convert_tokens_to_ids('[DOC]')

30522

In [ ]:
tokenizer.convert_tokens_to_ids('[PAT]')

30523

In [ ]:
len(tokenizer)


30526

In [ ]:
tokenizer.all_special_tokens

['[UNK]',
 '[SEP]',
 '[PAD]',
 '[CLS]',
 '[MASK]',
 '[DOC]',
 '[PAT]',
 '[SR_START]',
 '[SR_END]']

In [ ]:
tokenizer.all_special_ids


[1, 3, 0, 2, 4, 30522, 30523, 30524, 30525]

In [ ]:
tokenizer.all_special_tokens

['[UNK]',
 '[SEP]',
 '[PAD]',
 '[CLS]',
 '[MASK]',
 '[DOC]',
 '[PAT]',
 '[SR_START]',
 '[SR_END]']

In [ ]:
tokenizer.all_special_ids

[1, 3, 0, 2, 4, 30522, 30523, 30524, 30525]

In [ ]:
final_combined_all_dialog[0]

{'dialog': "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 'label': 'central retinal artery or vein occlusion'}

In [ ]:
len(final_combined_all_dialog)

1367

In [ ]:
temp_combined_list = []
for i in range(len(final_combined_all_dialog)):
  y = []
  y.append(final_combined_all_dialog[i]['dialog'])
  temp_label = final_combined_all_dialog[i]['label']
  temp_label_number = disease_to_id_small[temp_label]
  y.append(temp_label_number)
  temp_combined_list.append(y)

temp_combined_list[0]  


["[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 22]

In [ ]:
temp_combined_list[:3]

[["[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
  22],
 ["[PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SEP] [DOC] :  I can feel the pain you feel. Let me ask you some questions. Do you have Back pain? [SEP] [PAT] : Literally, I am suffering from Back pain. [SEP] [DOC] : Do you have Low back pain? [SEP] [PAT] : Yes, I am dealing with Low back pain. [SEP] [DOC] : Are you also dealing with Neck pain? [SEP] [PAT] : True, I have got Neck pain. [SEP

In [ ]:
len(temp_combined_list)

1367

In [ ]:
random.shuffle(temp_combined_list)




In [ ]:
temp_data1 = temp_combined_list[:957]
train_total_sep_dialog = []
train_total_dialog_label_id = []
for i in range(len(temp_data1)):
  # print(temp_data1[i][0]," \n ",temp_data1[i][1])
  temp_dlog = temp_data1[i][0]
  temp_label = temp_data1[i][1]
  # print("Dialog : ",temp_dlog)
  # print("Label : ",temp_label)

  train_total_sep_dialog.append(temp_dlog)
  train_total_dialog_label_id.append(temp_label)


  

In [ ]:
temp_data2 = temp_combined_list[957:]



len(temp_data2)

410

In [ ]:
valid_total_sep_dialog = []
valid_total_dialog_label_id = []

temp_data2_valid = temp_data2[:137]

temp_data2_test = temp_data2[137:]

for i in range(len(temp_data2_valid)):
  temp_valid_dlog = temp_data2_valid[i][0]
  temp_valid_label = temp_data2_valid[i][1]

  print("Dialog : ", temp_valid_dlog)
  print("Label : ", temp_valid_label)

  valid_total_sep_dialog.append(temp_valid_dlog)
  valid_total_dialog_label_id.append(temp_valid_label)

Dialog :  [PAT] : I have been experiencing disturbance of memory. Recently, I have noticed that I am asking the same questions repeatedly to the same person. Also, I tend to forget common words while talking. Is it something serious doctor? I am worried. [SEP] [DOC] : I understand how you feel. Let me do the symptom investigation first. Do you have depressive or psychotic symptoms? [SEP] [PAT] : Frequently. [SEP] [DOC] : You may have [MASK].
Label :  12
Dialog :  [PAT] : Hi doctor, I am struggling with Excessive urination at night. I am really concerned about it. I feel annoyed because of it. Please help. [SEP] [DOC] : I can understand what you are going through. It is a common feeling that comes when you experience a such issue. Do you feel Facial pain too? [SEP] [PAT] : That is true, doctor. [SEP] [DOC] : Are you having trouble with Joint stiffness or tightness? [SEP] [PAT] : Yes, I have a problem associated with Joint stiffness or tightness. [SEP] [DOC] : OK, that means you have [MA

In [ ]:
valid_total_sep_dialog[0]

'[PAT] : I have been experiencing disturbance of memory. Recently, I have noticed that I am asking the same questions repeatedly to the same person. Also, I tend to forget common words while talking. Is it something serious doctor? I am worried. [SEP] [DOC] : I understand how you feel. Let me do the symptom investigation first. Do you have depressive or psychotic symptoms? [SEP] [PAT] : Frequently. [SEP] [DOC] : You may have [MASK].'

In [ ]:
valid_total_dialog_label_id[0]

12

In [ ]:
temp_data2[1]

['[PAT] : Hi doctor, I am struggling with Excessive urination at night. I am really concerned about it. I feel annoyed because of it. Please help. [SEP] [DOC] : I can understand what you are going through. It is a common feeling that comes when you experience a such issue. Do you feel Facial pain too? [SEP] [PAT] : That is true, doctor. [SEP] [DOC] : Are you having trouble with Joint stiffness or tightness? [SEP] [PAT] : Yes, I have a problem associated with Joint stiffness or tightness. [SEP] [DOC] : OK, that means you have [MASK].',
 26]

In [ ]:
print(len(valid_total_sep_dialog))
print(len(valid_total_dialog_label_id))

137
137


In [ ]:
len(temp_data2_test)

273

In [ ]:
test_total_sep_dialog = []
test_total_dialog_label_id = []

for i in range(len(temp_data2_test)):
  temp_test_dialog = temp_data2_test[i][0]
  temp_test_label = temp_data2_test[i][1]

  # print("Dialog : ", temp_test_dialog)
  # print("Label : ", temp_test_label)

  test_total_sep_dialog.append(temp_test_dialog)
  test_total_dialog_label_id.append(temp_test_label)




In [ ]:
print(len(test_total_sep_dialog))
print(type(test_total_sep_dialog[0]))
print()
print(len(test_total_dialog_label_id))
print(type(test_total_dialog_label_id[0]))

273
<class 'str'>

273
<class 'int'>


In [ ]:
valid_total_sep_dialog[0]

'[PAT] : I have been experiencing disturbance of memory. Recently, I have noticed that I am asking the same questions repeatedly to the same person. Also, I tend to forget common words while talking. Is it something serious doctor? I am worried. [SEP] [DOC] : I understand how you feel. Let me do the symptom investigation first. Do you have depressive or psychotic symptoms? [SEP] [PAT] : Frequently. [SEP] [DOC] : You may have [MASK].'

In [ ]:
valid_total_sep_dialog[0]

'[PAT] : I have been experiencing disturbance of memory. Recently, I have noticed that I am asking the same questions repeatedly to the same person. Also, I tend to forget common words while talking. Is it something serious doctor? I am worried. [SEP] [DOC] : I understand how you feel. Let me do the symptom investigation first. Do you have depressive or psychotic symptoms? [SEP] [PAT] : Frequently. [SEP] [DOC] : You may have [MASK].'

In [ ]:
len(train_total_sep_dialog)

957

In [ ]:
len(train_total_dialog_label_id)

957

In [ ]:
len(valid_total_sep_dialog)

137

In [ ]:
len(valid_total_dialog_label_id)

137

In [ ]:
len(test_total_sep_dialog)

273

In [ ]:
len(test_total_dialog_label_id)

273

In [ ]:
type(train_total_sep_dialog)

list

In [ ]:
train_total_sep_dialog[0]

'[PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SEP] [DOC] : Per your symptom, you may have [MASK]. '

In [ ]:
train_total_sep_dialog[-1]

'[PAT] : Doctor, I am suffering from Itchiness of eye. And because of it, I cannot sleep properly. [SEP] [DOC] : Yes, itchiness can cause you trouble when you try to sleep. Do you have you got Pain in eye? [SEP] [PAT] : That is right, doctor. [SEP] [DOC] : Do you have Eye redness? [SEP] [PAT] : Right, I do have Eye redness. [SEP] [DOC] : Have you got Swollen eye? [SEP] [PAT] : Not sure. Please examine it, Swollen eye. [SEP] [DOC] : Your eye looks swelled up. Tell me, have you felt Painful sinuses? [SEP] [PAT] : You are right. I am having problem in breathing because of it. [SEP] [DOC] : I think you have [MASK].'

In [ ]:
len(train_total_sep_dialog)

957

In [ ]:
# valid_total_sep_dialog[0]

In [ ]:
input_ids_train_total_sep_dialog = []
token_type_ids_train_total_sep_dialog = []
attention_mask_train_total_sep_dialog = []

# enc = tokenizer(train_total_sep_dialog, padding='max_length', max_length = 512, return_tensors = 'pt')
enc = tokenizer(train_total_sep_dialog, padding='max_length', max_length = 512, truncation=True, return_tensors = 'pt')

input_ids_train_total_sep_dialog.append(enc['input_ids'])
token_type_ids_train_total_sep_dialog.append(enc['token_type_ids'])
attention_mask_train_total_sep_dialog.append(enc['attention_mask'])


print(enc.keys())
print(enc['input_ids'].shape)
print(input_ids_train_total_sep_dialog[0].shape)
print(input_ids_train_total_sep_dialog[0][:3,:].shape)
# h3 = bert_model(input_ids_train_total_sep_dialog[0][:3,:], attention_mask_train_total_sep_dialog[0][:3,:])
# print(type(h3))
# for i in range(len(train_total_sep_dialog)):
#     sent = train_total_sep_dialog[i]
#     enc = tokenizer(sent, padding='max_length', max_length = 512, return_tensors = 'pt')
# #     print(enc.keys())
#     input_ids_train_total_sep_dialog.append(enc['input_ids'])
#     token_type_ids_train_total_sep_dialog.append(enc['token_type_ids'])
#     attention_mask_train_total_sep_dialog.append(enc['attention_mask'])
print(enc['input_ids'][0,:10])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([957, 512])
torch.Size([957, 512])
torch.Size([3, 512])
tensor([    2, 30523,    30,  3091,  6012,     5, 11622,    18,    51,  2162])


In [ ]:
input_ids_valid_total_sep_dialog = []
token_type_ids_valid_total_sep_dialog = []
attention_mask_valid_total_sep_dialog = []


enc_valid = tokenizer(valid_total_sep_dialog, padding='max_length', max_length = 512, truncation=True, return_tensors = 'pt')

input_ids_valid_total_sep_dialog.append(enc_valid['input_ids'])
token_type_ids_valid_total_sep_dialog.append(enc_valid['token_type_ids'])
attention_mask_valid_total_sep_dialog.append(enc_valid['attention_mask'])

In [ ]:
t1_id = []
t1_mask = []
for i in range(len(train_total_sep_dialog)):
    print(enc['input_ids'].shape)
    print(enc['input_ids'][i].shape)
    print(enc['attention_mask'].shape)
    print(enc['attention_mask'][i].shape)
#     t1_id = 
    t1_id.append(enc['input_ids'][i])
    t1_mask.append(enc['attention_mask'][i])
    print()

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torch.Size([957, 512])
torch.Size([512])
torch.Size([957, 512])
torch.Size([512])

torc

In [ ]:

print(t1_id[0].shape)
len(t1_id)

torch.Size([512])


957

In [ ]:
print(t1_mask[0].shape)
len(t1_mask)

torch.Size([512])


957

In [ ]:
# print(h3.keys())

In [ ]:
# h3['last_hidden_state'].shape

In [ ]:
test_total_sep_dialog

["[PAT] : Doctor, I am having some issues. See, Skin rash [SEP] [DOC] : Have you ever noticed Abnormal appearing skin? [SEP] [PAT] : Yes Doctor. I have got some Abnormal appearing skin on my body. I am quite worried about this. [SEP] [DOC] : Please don't get tensed. Let me finish the symptom investigation first. Are you suffering from Skin moles? [SEP] [PAT] : Right, doctor. I am having quite a few Skin moles recently. [SEP] [DOC] : I can get that. Are you dealing with Lack of growth? [SEP] [PAT] : Yes Doctor.  [SEP] [DOC] : You might have [MASK]",
 "[PAT] : Hi Doctor, I am suffering from Shortness of breath. Due to it I am unable to breath properly, help me. [SEP] [DOC] : I'm sorry to hear this is troubling you. Can you say are you dealing with Rib pain too? [SEP] [PAT] : Yes Doctor. [SEP] [DOC] : Do you have Sharp chest pain? [SEP] [PAT] : I have Sharp chest pain Doctor. [SEP] [DOC] : Alright, are you suffering from Back pain? [SEP] [PAT] : Yes Doctor. It is causing a burning sensati

In [ ]:
input_ids_test_total_sep_dialog = []
token_type_ids_test_total_sep_dialog = []
attention_mask_test_total_sep_dialog = []

enc1 = tokenizer(test_total_sep_dialog, padding='max_length', max_length = 512, truncation=True, return_tensors = 'pt')
input_ids_test_total_sep_dialog.append(enc1['input_ids'])
token_type_ids_test_total_sep_dialog.append(enc1['token_type_ids'])
attention_mask_test_total_sep_dialog.append(enc1['attention_mask'])

print(enc1.keys())
print(enc1['input_ids'].shape)
print(len(input_ids_test_total_sep_dialog))
print(input_ids_test_total_sep_dialog[0].shape)

# for i in range(len(test_total_sep_dialog)):
#     sent = test_total_sep_dialog[i]
#     enc = tokenizer(sent, padding='max_length', max_length = 512, return_tensors = 'pt')
# #     print(enc.keys())
#     input_ids_test_total_sep_dialog.append(enc['input_ids'])
#     token_type_ids_test_total_sep_dialog.append(enc['token_type_ids'])
#     attention_mask_test_total_sep_dialog.append(enc['attention_mask'])


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([273, 512])
1
torch.Size([273, 512])


In [ ]:
enc1['input_ids'].shape

torch.Size([273, 512])

In [ ]:
# for i in range(10):
#   print(enc1['input_ids'][i,:])
#   print()

In [ ]:
t2_id = []
t2_mask = []

for i in range(len(test_total_sep_dialog)):
    print(enc1['input_ids'].shape)
    print(enc1['input_ids'][i].shape)
    print(enc1['attention_mask'].shape)
    print(enc1['attention_mask'][i].shape)
    
    t2_id.append(enc1['input_ids'][i])
    t2_mask.append(enc1['attention_mask'][i])
    print()

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torch.Size([273, 512])
torch.Size([512])
torch.Size([273, 512])
torch.Size([512])

torc

In [ ]:
print(t2_id[0].shape)
len(t2_id)

torch.Size([512])


273

In [ ]:
print(t2_mask[0].shape)
len(t2_mask)

torch.Size([512])


273

In [ ]:
print(len(input_ids_train_total_sep_dialog))
print(len(token_type_ids_train_total_sep_dialog))
print(len(attention_mask_train_total_sep_dialog))

1
1
1


In [ ]:
print(len(input_ids_test_total_sep_dialog))
print(len(token_type_ids_test_total_sep_dialog))
print(len(attention_mask_test_total_sep_dialog))

1
1
1


In [ ]:
input_ids_train_total_sep_dialog[0].shape

torch.Size([957, 512])

In [ ]:
input_ids_train_total_sep_dialog[0]

tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]])

In [ ]:
input_ids_train_total_sep_dialog[0].shape

torch.Size([957, 512])

In [ ]:
type(input_ids_train_total_sep_dialog)

list

In [ ]:
class DiseasePredictionDataset(Dataset):
#     X1 - (input id)

#     X2 - (attention mask)
#     Y - (labels)
    def __init__(self, X1, X2, Y):
#         self.X1 = X1
#         self.X2 = X2 
#         self.X3 = X3
#         self.X = [ X]
#         d1 = tokenizer(X, padding='max_length', max_length = 512, return_tensors = 'pt')
        self.X1 =  X1
        self.X2 = X2
        self.Y = Y
        
#         if(len(X1) != len(Y)):
#             raise Exception("The length of X does not match the length of Y")
#         elif(len(X2) != len(Y)):
#             raise Exception("The length of X2 does not match the length of Y")
#         if(len(X3) != len(Y)):
#             raise Exception("The length of X3 does not match the length of Y")    
        
    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, index):
        _x1 = self.X1[index]
        _x2 = self.X2[index]
#         _x3 = self.X3[index]
#         _x = self.X[index]
        _y = self.Y[index]
       
        
        return _x1, _x2, _y
    

In [ ]:
type(train_total_dialog_label_id)

list

In [ ]:
train_total_dialog_label_id = torch.tensor(train_total_dialog_label_id)
type(train_total_dialog_label_id)

torch.Tensor

In [ ]:
train_total_dialog_label_id.shape

torch.Size([957])

In [ ]:
#  need to convert valid dialog id into tensor

In [ ]:
valid_total_dialog_label_id = torch.tensor(valid_total_dialog_label_id)

In [ ]:
type(valid_total_dialog_label_id)

torch.Tensor

In [ ]:
test_total_dialog_label_id = torch.tensor(test_total_dialog_label_id)
type(test_total_dialog_label_id)

torch.Tensor

In [ ]:
type(train_total_dialog_label_id)

torch.Tensor

In [ ]:
train_loader = DataLoader(DiseasePredictionDataset(enc['input_ids'],  enc['attention_mask'], train_total_dialog_label_id), batch_size=16, shuffle = True)
valid_loader = DataLoader(DiseasePredictionDataset(enc_valid['input_ids'], enc_valid['attention_mask'], valid_total_dialog_label_id), batch_size=16, shuffle = False)
test_loader = DataLoader(DiseasePredictionDataset(enc1['input_ids'], enc1['attention_mask'], test_total_dialog_label_id), batch_size = 16, shuffle=False)

# train_loader = DataLoader(DiseasePredictionDataset(enc['input_ids'],  enc['attention_mask'], train_total_dialog_label_id), batch_size=16)
# test_loader = DataLoader(DiseasePredictionDataset(enc1['input_ids'], enc1['input_ids'], test_total_dialog_label_id), batch_size = 16)


# train_loader = DataLoader(DiseasePredictionDataset(train_total_sep_dialog, train_total_dialog_label_id), batch_size=128,shuffle=True)
# test_loader = DataLoader(DiseasePredictionDataset(test_total_sep_dialog, test_total_dialog_label_id), batch_size = 128,shuffle=True)




In [ ]:
train_loader

In [ ]:
valid_loader

In [ ]:
test_loader

In [ ]:
sm = 0
for i,j in enumerate(train_loader):
    print(i)
#     print(type(j))
#     print(len(j))
    # print(type(j[0]))

#     print(len(j[0]))
#     print(type(j[1]))
#     print(len(j[1]))
#     print(j)
#     print(j[0])
#     print(len(j[0]))
    # sm += len(j[0])
#     print(j[0])
#     break
    # print()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [ ]:
sm = 0
for i,j in enumerate(test_loader):
    print(i)
#     print(type(j))
#     print(len(j))
#     print(type(j[0]))

#     print(len(j[0]))
#     print(type(j[1]))
#     print(len(j[1]))
#     print(j)
#     print(j[0])
#     print(len(j[0]))
    # sm += len(j[0])
#     print(j[0])
#     break
    # print()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [ ]:
# sm

In [ ]:
sm = 0
for i,j in enumerate(test_loader):

    print(len(j[0]))
    sm += len(j[0])
    print()

16

16

16

16

16

16

16

16

16

16

16

16

16

16

16

16

16

1



In [ ]:
sm

273

In [ ]:
for i in train_loader:
#     print(len(i))
#     print(i[0].shape)
#     print(i[-1])
#     s = i[1]
#     print(s)
    print(type(i[0]))
    print(type(i[1]))
    print(type(i[2]))
#     print(type(i[3]))
    print()

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>

<class 'torch.Tensor

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class CustomBert(torch.nn.Module):
    def __init__(self):
        super(CustomBert, self).__init__()
        self.bert = bert_model
        
        self.linear1 = torch.nn.Linear(768, 256)
        self.linear2 = torch.nn.Linear(256,90)
        
    def forward(self, ids, mask):
        # print('Hello_1')
        output_encoding = self.bert(ids, mask)
        # print('Hello_2')
        # print()
        cls = output_encoding['last_hidden_state'] .permute(1,0,2)[0]
        
        linear_output1 = self.linear1(cls)
        linear_output2 = self.linear2(F.relu(linear_output1))
        
        return linear_output2
#         return output_encoding['last_hidden_state'].shape
        

In [ ]:
model = CustomBert()
model

CustomBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30526, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (d

In [ ]:
model = model.to(device)
model

CustomBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30526, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (d

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7ff62ae7a580>

In [ ]:
# total_params = sum(p.numel() for p in model.parameters())
# total_params

In [ ]:
# pytorch_total_params = sum(p.numel() for p in model.parameters())
# pytorch_total_params/1e6

In [ ]:
total_param = 0
for p in model.parameters():
    total_param += p.numel()


In [ ]:
total_param/1e6

109.705306

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [ ]:
for i,name in enumerate(model.named_parameters()):
    print(i, ' ',name[0], ' ', len(name))

0   bert.embeddings.word_embeddings.weight   2
1   bert.embeddings.position_embeddings.weight   2
2   bert.embeddings.token_type_embeddings.weight   2
3   bert.embeddings.LayerNorm.weight   2
4   bert.embeddings.LayerNorm.bias   2
5   bert.encoder.layer.0.attention.self.query.weight   2
6   bert.encoder.layer.0.attention.self.query.bias   2
7   bert.encoder.layer.0.attention.self.key.weight   2
8   bert.encoder.layer.0.attention.self.key.bias   2
9   bert.encoder.layer.0.attention.self.value.weight   2
10   bert.encoder.layer.0.attention.self.value.bias   2
11   bert.encoder.layer.0.attention.output.dense.weight   2
12   bert.encoder.layer.0.attention.output.dense.bias   2
13   bert.encoder.layer.0.attention.output.LayerNorm.weight   2
14   bert.encoder.layer.0.attention.output.LayerNorm.bias   2
15   bert.encoder.layer.0.intermediate.dense.weight   2
16   bert.encoder.layer.0.intermediate.dense.bias   2
17   bert.encoder.layer.0.output.dense.weight   2
18   bert.encoder.layer.0.output

In [ ]:
cnt = 0
for name, param in model.named_parameters():
    if(param.requires_grad == False and cnt >= 199):
        print(name)
        param.requires_grad=True
    cnt += 1

linear1.weight
linear1.bias
linear2.weight
linear2.bias


In [ ]:
total_train_param = 0
for p in model.parameters():
    print(p.requires_grad)
    if p.requires_grad==True:
        total_train_param += p.numel()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [ ]:
total_train_param/1e6

0.219994

In [ ]:
for i in train_loader:
#     print(i[0])
    print(i[1])

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
train_loader

In [ ]:
len(train_loader.dataset)

957

In [ ]:
for k,j in enumerate(train_loader):
  print(len(j))
  print(j[0].shape)
  break

3
torch.Size([16, 512])


In [ ]:
for i,batch in enumerate(train_loader):
  print(batch[2])

tensor([71, 13, 34, 38, 53, 34, 29, 16, 38, 46, 51, 17, 72, 40, 31, 63])
tensor([48, 33, 80, 76, 77,  4, 58, 57, 16, 51, 40, 43,  0, 21, 64, 61])
tensor([25, 57, 76, 40, 71, 72, 13,  2, 11, 12, 63, 62, 46, 46, 35, 54])
tensor([58, 76, 23, 60, 30, 66, 37, 63, 76, 83, 35, 83, 15, 14, 47, 41])
tensor([68,  6, 20, 29, 42, 40, 23, 79, 40, 12, 45, 86, 62, 75, 20, 24])
tensor([31, 68, 73, 34, 45, 56,  7, 46, 55,  6, 22, 23, 24, 73, 27, 74])
tensor([29, 56,  1, 47, 57, 88, 10, 89, 53, 74, 27, 71, 62, 16, 12, 20])
tensor([82, 52, 60, 62, 69,  5, 27, 60,  6,  7,  4, 46, 69, 28, 47, 76])
tensor([63, 52, 80, 79,  3, 14, 10, 21, 37,  9, 69, 59, 30, 67, 11,  2])
tensor([82, 36, 28, 10, 63, 29, 25, 64,  1,  2, 38, 87, 48, 77, 73, 44])
tensor([57, 38, 14,  7, 17, 79, 69, 47, 71, 19, 29, 36, 60, 23, 29,  2])
tensor([45, 51, 78, 89,  1, 76, 67, 42, 39,  0, 77, 28, 52, 51, 63, 27])
tensor([64, 74, 25, 25, 66, 79, 79, 27, 66, 89, 88, 14, 85, 57, 82, 62])
tensor([63, 26, 27, 33, 44, 88, 67, 67, 79, 37, 27,

In [ ]:
for i,batch in enumerate(valid_loader):
  print(batch[2])

tensor([12, 26, 71, 18, 20, 19, 42, 33, 45, 76, 44, 61,  5, 68, 70, 61])
tensor([55, 15, 47, 45, 55, 88, 19, 65,  2,  2, 52,  2, 40, 28, 77, 16])
tensor([51,  5, 68,  5, 13, 11, 45, 14, 88, 22,  1, 71, 53,  2, 76, 39])
tensor([47, 57, 47,  2, 58, 58, 12, 72, 58, 10, 69, 40,  5, 18, 16, 46])
tensor([62, 58, 65, 19, 89, 78, 78,  3, 41, 86, 13, 28, 21, 46, 26, 51])
tensor([28, 81,  5, 56, 34, 42, 62, 32, 28, 41, 58, 13, 16, 47, 33, 49])
tensor([26, 38,  1, 75, 80, 11,  9, 81, 22, 55, 24,  1, 83, 32, 73,  7])
tensor([24, 13, 35, 25, 69, 77, 23, 44, 14, 24, 48, 70, 23,  4, 27, 55])
tensor([ 0, 80, 39, 56,  5, 17, 78, 44, 22])


In [ ]:
def train(loader):
    model.train()
    
    for i,batch in enumerate(loader):
        # print(type(batch))
        # batch = batch.to(device)
        st_time = time.time()
        optimizer.zero_grad()
#         data = batch[0]
#         target = batch[1]
#         tokenizer_output = tokenizer.batch_encode_plus(data, return_tensors='pt', padding = True)
        input_ids = torch.tensor(batch[0]).to(device)
        attention_mask = torch.tensor(batch[1]).to(device)
        target = torch.tensor(batch[2]).to(device)
#         target = batch[3]
#         tokenizer_output = {}
#         tokenizer_output['input_ids'] = input_ids
#         tokenizer_output['token_type_ids'] = token_type_ids
#         tokenizer_output['attention_mask'] = attention_mask
#         outputs = model(tokenizer_output)
#         outputs = model(input_ids, mask = attention_mask)
#         outputs = model(input_ids)
        # print('Input : ',input_ids.shape)
#         print(input_ids )
#         print('Attention mask : ',attention_mask.shape)
#         print(attention_mask)
#         e6 = bert_model(input_ids, attention_mask)
#         print(e6.keys())
#         print()
        outputs = model(input_ids, attention_mask)
        outputs = F.log_softmax(outputs, dim =1)
#         output = 
        loss = criterion(outputs, target)
        if(i%20 == 0):
            print('Loss : ', loss)
            
        
        loss.backward()
        optimizer.step()
        
        end_time = time.time() - st_time
        # print('Per Batch time  : {}'.format(round(end_time,3)))
        # print()

In [ ]:
# end_time = 4.6768
# print('Per Batch time  : {}'.format(round(end_time,3)))

In [ ]:
len(train_loader.dataset)

957

In [ ]:
def test(loader):
    model.eval()
    
    y_true = []
    y_pred = []

    correct = 0
    for i, batch in enumerate(loader):
        with torch.no_grad():
            # batch = batch.to(device)


            input_ids = torch.tensor(batch[0]).to(device)

            attention_mask = torch.tensor(batch[1]).to(device)
            target = torch.tensor(batch[2]).to(device)
    #         target = batch[3]
    #         tokenizer_output = {}
    #         tokenizer_output['input_ids'] = input_ids
    #         tokenizer_output['token_type_ids'] = token_type_ids
    #         tokenizer_output['attention_mask'] = attention_mask
    #         print(input_ids.shape)
    #         print(attention_mask.shape)
            # outputs = model(input_ids, mask = attention_mask)
    #         outputs = model(input_ids)
            
            outputs = model(input_ids, attention_mask)
            # outputs = F.log_softmax(outputs, dim =1)
            pred = outputs.argmax(dim=1)
            
    #         output = 
            loss = criterion(outputs, target)
            if((len(loader.dataset) < 977)):
                # print('Input_ids : ', input_ids)
                # print('Attention_mask : ', attention_mask)
                # print('Prediction : ', outputs)
                print('Prediction label : ', pred)
                print('Ground : ', target)
                print('Loss : ', loss)
            
    #         pred = 0
            # print('Prediction : ', pred)
            # print('Truth : ', target)
            # print()
            correct += int((pred == target).sum())

            y_true.extend(target.tolist())
            y_pred.extend(pred.tolist())

            
    
    return correct/len(loader.dataset), y_true, y_pred
        

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device

In [ ]:

# model = model.to(device)

In [ ]:
def train_and_validation(model, train_loader, valid_loader):
  best_acc = 0.0
  

  for epoch in range(25):

    print("\n=============Epoch : ", epoch)
    train( train_loader)
    # valid_acc, valid_pred, valid_gold = test(valid_loader)
    valid_acc, y_true_valid, y_pred_valid = test(valid_loader)

    print("Length of predictions : ", len(y_pred_valid))
    print("Length of gold : ", len(y_true_valid))
    # print("Valid loss : ", valid_loss)
    
    print("\nAccuracy\n", accuracy_score(y_true_valid, y_pred_valid))
    print("\nF1 score\n", f1_score(y_true_valid, y_pred_valid, average = 'macro'))
    print("\nJaccard similarity\n", jaccard_score(y_true_valid, y_pred_valid, average = 'macro')) 

    curr_acc = accuracy_score(y_true_valid, y_pred_valid)

    print('Curr acc : ', curr_acc)
    print('Best acc : ', best_acc)

    if(curr_acc > best_acc):
      best_acc = curr_acc

      
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/models/dialog_only/best_model_epoch_'+str(epoch)+'_best_acc_'+str(int(best_acc*100))+'_.pt')

      print("model saved\n")

In [ ]:
train_and_validation(model, train_loader, valid_loader)


=============Epoch :  0
Loss :  tensor(3.1748, device='cuda:0', grad_fn=<NllLossBackward0>)


<ipython-input-114-aea576cbb494>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch[0]).to(device)
<ipython-input-114-aea576cbb494>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch[1]).to(device)
<ipython-input-114-aea576cbb494>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(batch[2]).to(device)


Loss :  tensor(2.8038, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss :  tensor(2.3937, device='cuda:0', grad_fn=<NllLossBackward0>)


<ipython-input-117-29b4fa65a869>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch[1]).to(device)
<ipython-input-117-29b4fa65a869>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(batch[2]).to(device)


Prediction label :  tensor([12, 75, 58, 73, 20, 28,  7, 20, 45, 76, 44, 61, 23, 68, 68, 61],
       device='cuda:0')
Ground :  tensor([12, 26, 71, 18, 20, 19, 42, 33, 45, 76, 44, 61,  5, 68, 70, 61],
       device='cuda:0')
Loss :  tensor(2.0027, device='cuda:0')
Prediction label :  tensor([52, 48, 59, 45, 27, 31, 53, 64,  2,  2, 52,  2, 61, 28, 77,  6],
       device='cuda:0')
Ground :  tensor([55, 15, 47, 45, 55, 88, 19, 65,  2,  2, 52,  2, 40, 28, 77, 16],
       device='cuda:0')
Loss :  tensor(2.0035, device='cuda:0')
Prediction label :  tensor([44, 23, 68, 23, 44, 58, 45, 73,  2, 54, 27, 58, 53,  2, 64, 69],
       device='cuda:0')
Ground :  tensor([51,  5, 68,  5, 13, 11, 45, 14, 88, 22,  1, 71, 53,  2, 76, 39],
       device='cuda:0')
Loss :  tensor(2.3544, device='cuda:0')
Prediction label :  tensor([47,  6,  2,  2, 23, 38, 60, 73, 62, 10, 35, 61, 44, 23,  6, 46],
       device='cuda:0')
Ground :  tensor([47, 57, 47,  2, 58, 58, 12, 72, 58, 10, 69, 40,  5, 18, 16, 46],
       d

...

Prediction label :  tensor([12, 26, 71, 18, 20, 59,  7, 33, 45, 76, 44, 40,  5, 68, 70, 61],
       device='cuda:0')
Ground :  tensor([12, 26, 71, 18, 20, 19, 42, 33, 45, 76, 44, 61,  5, 68, 70, 61],
       device='cuda:0')
Loss :  tensor(0.4446, device='cuda:0')
Prediction label :  tensor([55, 30, 47, 45, 55, 31, 53, 64,  2,  2, 52, 27, 40, 28, 77,  6],
       device='cuda:0')
Ground :  tensor([55, 15, 47, 45, 55, 88, 19, 65,  2,  2, 52,  2, 40, 28, 77, 16],
       device='cuda:0')
Loss :  tensor(1.0972, device='cuda:0')
Prediction label :  tensor([51, 51, 68,  5,  5, 71, 45, 51, 88, 17, 27, 71, 53, 38, 64, 39],
       device='cuda:0')
Ground :  tensor([51,  5, 68,  5, 13, 11, 45, 14, 88, 22,  1, 71, 53,  2, 76, 39],
       device='cuda:0')
Loss :  tensor(1.5806, device='cuda:0')
Prediction label :  tensor([ 0, 57, 47, 27, 88, 58, 10, 73, 58, 10, 69, 41,  5, 18,  6, 46],
       device='cuda:0')
Ground :  tensor([47, 57, 47,  2, 58, 58, 12, 72, 58, 10, 69, 40,  5, 18, 16, 46],
       d

<ipython-input-114-aea576cbb494>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch[0]).to(device)
<ipython-input-114-aea576cbb494>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch[1]).to(device)
<ipython-input-114-aea576cbb494>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(batch[2]).to(device)


Loss :  tensor(0.0537, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss :  tensor(0.1114, device='cuda:0', grad_fn=<NllLossBackward0>)


<ipython-input-117-29b4fa65a869>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch[1]).to(device)
<ipython-input-117-29b4fa65a869>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(batch[2]).to(device)


Prediction label :  tensor([12, 26, 71, 18, 20, 59,  7, 33, 45, 76, 44, 54,  5, 68, 70, 61],
       device='cuda:0')
Ground :  tensor([12, 26, 71, 18, 20, 19, 42, 33, 45, 76, 44, 61,  5, 68, 70, 61],
       device='cuda:0')
Loss :  tensor(0.5147, device='cuda:0')
Prediction label :  tensor([55, 34, 47, 45, 55, 31, 53, 64,  2,  2, 52,  2, 67, 85, 77, 16],
       device='cuda:0')
Ground :  tensor([55, 15, 47, 45, 55, 88, 19, 65,  2,  2, 52,  2, 40, 28, 77, 16],
       device='cuda:0')
Loss :  tensor(1.0776, device='cuda:0')
Prediction label :  tensor([44,  5, 68,  5,  5, 71, 45, 51, 88, 54, 27, 71, 53, 38, 64, 39],
       device='cuda:0')
Ground :  tensor([51,  5, 68,  5, 13, 11, 45, 14, 88, 22,  1, 71, 53,  2, 76, 39],
       device='cuda:0')
Loss :  tensor(1.5860, device='cuda:0')
Prediction label :  tensor([ 0, 57, 47, 27, 31, 58, 69, 73, 11, 10, 69, 41,  5, 18,  6, 46],
       device='cuda:0')
Ground :  tensor([47, 57, 47,  2, 58, 58, 12, 72, 58, 10, 69, 40,  5, 18, 16, 46],
       d

In [ ]:
# for epoch in range(25):
#     train(train_loader)
#     # print('=============================================Its Training Prediction================================\n')
#     train_acc, y_true_train, y_pred_train  = test(train_loader)
#     # print('=============================================Its Validation Prediction================================\n')
#     valid_acc, y_true_valid, y_pred_valid = test(valid_loader)
#     print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Valid Acc : {valid_acc:.4f}')

#     # test_acc = test(test_loader)
#     # print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc : {test_acc:.4f}')

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/models/dialog_only/best_model_epoch_16_best_acc_62_.pt'

In [ ]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
test_acc, y_true_test, y_pred_test = test(test_loader)
print(f'Test Acc : {test_acc:.4f}')

<ipython-input-117-29b4fa65a869>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch[0]).to(device)
<ipython-input-117-29b4fa65a869>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch[1]).to(device)
<ipython-input-117-29b4fa65a869>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(batch[2]).to(device)


Prediction label :  tensor([27, 82, 79, 29,  6, 48, 58, 76, 25, 39, 29, 54,  2, 27, 59,  3],
       device='cuda:0')
Ground :  tensor([50, 82, 86, 29, 16, 78, 58, 76, 43, 39, 29, 17,  2, 50, 59, 50],
       device='cuda:0')
Loss :  tensor(1.3964, device='cuda:0')
Prediction label :  tensor([27, 54, 54, 56, 88, 20, 87, 79, 30, 70, 67, 63, 59, 65, 69, 60],
       device='cuda:0')
Ground :  tensor([ 1, 54, 54, 10, 88, 20, 87, 86, 48, 70, 67, 63, 59, 65, 69, 60],
       device='cuda:0')
Loss :  tensor(1.2088, device='cuda:0')
Prediction label :  tensor([61, 72,  5, 61, 83, 50, 76, 62,  2, 60, 16, 70, 17, 32,  9,  7],
       device='cuda:0')
Ground :  tensor([54, 73,  5, 17,  0, 50, 66, 59,  2, 51, 16, 70, 54, 78,  9, 26],
       device='cuda:0')
Loss :  tensor(2.4457, device='cuda:0')
Prediction label :  tensor([24, 72, 15,  8, 82, 64, 15, 62, 38,  6, 15, 48, 56, 70, 22, 67],
       device='cuda:0')
Ground :  tensor([46, 72, 32,  8, 82, 64, 32, 38, 38,  6, 15, 48,  8, 70, 22, 40],
       d

In [ ]:
print("=========Using Sklearn======")
print("\nConfusion matrix\n", confusion_matrix(y_true_test, y_pred_test))
print("\nAccuracy\n", accuracy_score(y_true_test, y_pred_test))
print("\nF1 score\n", f1_score(y_true_test, y_pred_test, average = 'macro'))
print("\nJaccard similarity\n", jaccard_score(y_true_test, y_pred_test, average = 'macro'))

=========Using Sklearn======

Confusion matrix
 [[0 0 2 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 2 1]
 [0 0 0 ... 0 0 2]]

Accuracy
 0.5897435897435898

F1 score
 0.5306743761289217

Jaccard similarity
 0.43318452380952377


In [ ]:
# {'input_ids': tensor([[    2, 30523,    30, 11622,    16,    51,  3365, 10950,  2014, 15321,
#           1036,  1922,  3057,  9266,    18,    51,  2112,    11,    62,  3365,
#          22491,  5436,    18,    51,  9248,  4087,  5427,    18,     3, 30522,
#             30,    51,  5734,  2307,  6253,  9248,    18,    51,  2942,  1942,
#           5839,    43,  3855,  5610,  1942, 15800, 10186,  2573,    18,  2162,
#           6253, 14643,  9597,  9266,    35,     3, 30523,    30,  6228,    16,
#             51,  2150,  2222, 17463,  1956,  9597,  9266,    18,     3, 30522,
#             30,  2608,  6253,  2162,  3601,  1927,  5723,    35,     3, 30523,
#             30, 11633,    16,    51,  2162, 14643,  3601,  1927,  5723,    18,
#              3, 30522,    30,  2162,  6253, 11272,  3594,  1922,  5723,    35,
#              3, 30523,    30,  5087,    16,    51,  2150, 11364,  2037,  3594,
#           1922,  5723,    18,     3, 30522,    30, 10186,  3601,  3579,  1988,
#           6253,  2162,     4,    18,     3,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
#              0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#          0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1